# AIDO.Cell Quickstart

This demo quickly walks through installing AIDO.Cell and embedding new single cell data.

__Requirements__:
- A100 GPU or equivalent

### Install ModelGenerator and extra dependencies for tutorials

In [ ]:
!git clone https://github.com/genbio-ai/ModelGenerator.git
%cd ModelGenerator
!pip install -e ".[flash_attn]"
%cd experiments/AIDO.Cell
!pip install -r requirements.txt

In [ ]:
# Restart the session after installing

# Then navigate back to the AIDO.Cell directory
%cd ModelGenerator/experiments/AIDO.Cell

### Grab some data from GEO and load into anndata

In [ ]:
%%bash
mkdir -p data
cd data
wget -nv -O GSE214695.tar 'http://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE214695&format=file'
tar -xvf GSE214695.tar
cd ..

In [1]:
import anndata as ad
import scanpy as sc

adata = sc.read_10x_mtx('data', prefix='GSM6614348_HC-1_')
sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_genes(adata, min_cells=3)
# No more normalization needed, AIDO.Cell uses raw counts

### Preprocess the anndata for AIDO.Cell

In [2]:
import cell_utils
aligned_adata, attention_mask = cell_utils.align_adata(adata)

###########  Aligning data to AIDO.Cell  ###########
AIDO.Cell was pretrained on a fixed set of 19264 genes.
Aligning your data to the AIDO.Cell gene set...
2428 in your data that cannot be used by AIDO.Cell. Removing these.
['A1BG-AS1' 'A2M-AS1' 'AAED1' ... 'ZNRD1' 'ZNRF3-AS1' 'ZSCAN16-AS1']
5837 genes in the AIDO.Cell pretraining set missing in your data.
AIDO.Cell is trained with zero-masking. Setting these to zero for AIDO.Cell to ignore.
['A2ML1' 'A3GALT2' 'A4GNT' ... 'ZSWIM5' 'ZYG11A' 'ZZZ3']
13427 non-zero genes remaining.
Reordering genes to match AIDO.Cell gene ordering
Gathering attention mask for nonzero genes
####################  Finished  ####################


### Get AIDO.Cell embeddings

AIDO.Cell was designed for use with the ModelGenerator CLI. 
It is strongly recommended to use ModelGenerator for using the AIDO.Cell model.
For more information check out
- [Using ModelGenerator to finetune AIDO.Cell](./tutorial_cell_classification.ipynb)
- [ModelGenerator Docs](https://genbio-ai.github.io/ModelGenerator/)

In [ ]:
# Embed
import anndata as ad
import numpy as np
import torch
import sys
from modelgenerator.tasks import Embed

# The following is equivalent to the ModelGenerator CLI command:
# mgen predict --model Embed --model.backbone aido_cell_3m \
#   --data CellClassificationDataModule --data.test_split_files <your_anndata>.h5ad

# If not using mgen, this should be configured manually.
device = 'cuda'
batch_size = 2

model = Embed.from_config({
        "model.backbone": "aido_cell_3m",
        "model.batch_size": batch_size
    }).eval()
model = model.to(device).to(torch.bfloat16)

# All data must be in bfloat16
batch_np = aligned_adata[:batch_size].X.toarray()
batch_tensor = torch.from_numpy(batch_np).to(torch.bfloat16).to(device)
# Call transform and embed.
batch_transformed = model.transform({'sequences': batch_tensor})
embs = model(batch_transformed)

# Full Embeddings
print('FULL EMBEDDING')
print('(batch_size, genes, embedding_dim)')
print(embs.shape)
print(embs)
print('-------------------------------------')

# Non-Zero Genes Embeddings
print('NON-ZERO GENES EMBEDDING')
embs = embs[:, attention_mask.astype(bool), :]
print('(batch_size, genes, embedding_dim)')
print(embs.shape)
print(embs)

/home/caleb_ellington_genbio_ai/miniforge3/envs/mgen/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/caleb_ellington_genbio_ai/ModelGenerator/modelgenerator/tasks/tasks.py:1667: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids, dtype=torch.long).to(self.device)
/home/caleb_ellington_genbio_ai/ModelGenerator/modelgenerator/backbones/backbones.py:672: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(


FULL EMBEDDING
(batch_size, genes, embedding_dim)
torch.Size([2, 19264, 128])
tensor([[[-2.0469,  0.4199, -1.6719,  ..., -0.9258,  0.3730,  1.5938],
         [-0.6445, -1.9062, -2.7969,  ..., -1.5391,  0.9414, -0.5273],
         [-1.0703, -1.5234, -0.9648,  ..., -0.6445,  0.6406,  0.8867],
         ...,
         [ 0.5586, -1.8672, -2.6562,  ..., -0.3438, -0.2100,  0.9297],
         [ 0.0037,  0.0347,  0.2969,  ..., -0.4258,  1.3438, -0.4121],
         [-1.1172, -1.5156, -1.0781,  ..., -1.0781,  1.4531, -0.9727]],

        [[-2.3125,  1.0391, -2.3125,  ..., -0.2471,  0.5312,  0.1572],
         [-0.8008, -2.0000, -2.7344,  ..., -1.4688,  0.6328, -0.7422],
         [-0.0918, -2.2188, -0.0815,  ..., -1.4453,  0.0179,  0.8438],
         ...,
         [ 0.0698, -1.3359, -2.4375,  ..., -0.0195,  0.0396,  1.0547],
         [ 0.1777,  0.0664,  0.3223,  ..., -0.1631,  1.0938, -0.3145],
         [-0.6602, -1.0000, -1.5469,  ..., -1.0312,  0.9883, -0.7266]]],
       device='cuda:0', dtype=torch.bf